# Projeto 1 - Redes Sociais

## Alunos

<table>
  <tr>
    <td align="center"><a href="https://github.com/DaviReisVieira"><img style="border-radius: 50%;" src="https://github.com/DaviReisVieira.png" width="100px;" alt=""/><br /><sub><b>Davi Reis Vieira</b></sub></a><br /><a href="https://github.com/DaviReisVieira" title="Davi Reis Vieira">O Cara -1</a></td>
    <td align="center"><a href="https://github.com/guishas"><img style="border-radius: 50%;" src="https://github.com/guishas.png" width="100px;" alt=""/><br /><sub><b>Guilherme Lunetta</b></sub></a><br /><a href="https://github.com/guishas" title="Guilherme Lunetta">O cara 0</a></td>
    <td align="center"><a href="https://github.com/josermf2"><img style="border-radius: 50%;" src="https://github.com/josermf2.png" width="100px;" alt=""/><br /><sub><b>José Fernandes</b></sub></a><br /><a href="https://github.com/josermf2" title="José Fernandes">O cara +1</a></td>
  </tr>
</table>

# Classificador de Reviews

- Importando bibliotecas

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('reviews_april.csv')
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,68330832,brazilian,"Jogo excelente em todas as mecânicas, gráficos...",2020-04-30 23:52:54,2020-04-30 23:52:54,True,0,0,...,True,False,False,76561198159239276,26,2,9640.0,0.0,9009.0,1.588377e+09
1,292030,The Witcher 3: Wild Hunt,68329511,brazilian,o jogo é pika,2020-04-30 23:19:24,2020-04-30 23:19:24,True,0,0,...,True,False,False,76561198043602884,34,6,10299.0,0.0,2568.0,1.606605e+09
2,292030,The Witcher 3: Wild Hunt,68328796,brazilian,Nice,2020-04-30 23:01:10,2020-04-30 23:01:10,True,0,0,...,True,False,False,76561198134272383,21,5,783.0,0.0,619.0,1.609530e+09
3,292030,The Witcher 3: Wild Hunt,68327598,brazilian,muito bom,2020-04-30 22:30:41,2020-04-30 22:30:41,True,0,0,...,True,False,False,76561199006967739,12,2,5922.0,0.0,5533.0,1.590269e+09
4,292030,The Witcher 3: Wild Hunt,68325545,brazilian,"Temática e história ricas de detalhes, um bom ...",2020-04-30 21:42:55,2020-04-30 21:42:55,True,0,0,...,True,False,False,76561198062670500,36,1,5192.0,0.0,1514.0,1.589417e+09


In [25]:
!pip install --upgrade "ibm-watson>=6.1.0"

You should consider upgrading via the 'c:\users\vieir\desktop\insper\6-semestre\redes-sociais\projeto 1\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions

def analise_sentimento(texto):
    authenticator = IAMAuthenticator('o8Sc5-2wq7yCwjhF9QIqVaMkQC-Y_zS2rodG1yzIe6r9')
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2022-04-07',
        authenticator=authenticator
    )
    natural_language_understanding.set_service_url('https://api.eu-de.natural-language-understanding.watson.cloud.ibm.com/instances/6d4f1ad2-a676-46bb-829f-eb45a3856fb0')
    response = natural_language_understanding.analyze(
        text=texto,
        features=Features(sentiment=SentimentOptions())).get_result()
    print(response)
    return response

In [7]:
# load 'sentimentos.json'
import json

with open('sentimentos.json') as json_file:
    sentimentos_load = json.load(json_file)

In [8]:
def pool_function(array):
    if array['score'] == None:
        try:
            review_text = df[df['review_id'] == array['review_id']]['review'].values[0]
            response = analise_sentimento(array['review'])
            score = response['sentiment']['document']['score']
            label = response['sentiment']['document']['label']
            return {'review_id': array['review_id'],'score': score, 'label': label}
        except:
            return {'review_id': array['review_id'],'score': None, 'label': None}
    else:
        return array

In [12]:
from multiprocessing import Pool

with Pool(50) as p:
    sentimentos = p.map(pool_function, sentimentos_load)

# save 'sentimentos.json'
with open('sentimentos2.json', 'w') as outfile:
    json.dump(sentimentos, outfile)

In [39]:
# cria colunas score e label no dataframe
df_sentimentos = pd.DataFrame(sentimentos)
df = df.merge(df_sentimentos, on='review_id')
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,score,label
0,292030,The Witcher 3: Wild Hunt,68330832,brazilian,"Jogo excelente em todas as mecânicas, gráficos...",2020-04-30 23:52:54,2020-04-30 23:52:54,True,0,0,...,False,76561198159239276,26,2,9640.0,0.0,9009.0,1.588377e+09,0.984265,positive
1,292030,The Witcher 3: Wild Hunt,68329511,brazilian,o jogo é pika,2020-04-30 23:19:24,2020-04-30 23:19:24,True,0,0,...,False,76561198043602884,34,6,10299.0,0.0,2568.0,1.606605e+09,NaN,None
2,292030,The Witcher 3: Wild Hunt,68328796,brazilian,Nice,2020-04-30 23:01:10,2020-04-30 23:01:10,True,0,0,...,False,76561198134272383,21,5,783.0,0.0,619.0,1.609530e+09,NaN,None
3,292030,The Witcher 3: Wild Hunt,68327598,brazilian,muito bom,2020-04-30 22:30:41,2020-04-30 22:30:41,True,0,0,...,False,76561199006967739,12,2,5922.0,0.0,5533.0,1.590269e+09,NaN,None
4,292030,The Witcher 3: Wild Hunt,68325545,brazilian,"Temática e história ricas de detalhes, um bom ...",2020-04-30 21:42:55,2020-04-30 21:42:55,True,0,0,...,False,76561198062670500,36,1,5192.0,0.0,1514.0,1.589417e+09,0.954308,positive
